In [1]:
import os

In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories=['with_mask','without_mask']

In [5]:
data=[]
for category in categories:
    path=os.path.join('train',category)
    
    label=categories.index(category)
    
    for file in os.listdir(path):
        
        img_path=os.path.join(path,file)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])

In [6]:
len(data)

2712

In [7]:
import random

In [8]:
random.shuffle(data)

In [9]:
x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)

In [10]:
len(x)


2712

In [11]:
len(y)

2712

In [12]:
import numpy as np
x=np.array(x)
y=np.array(y)

In [13]:
x=x/255

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [15]:
x_train.shape

(2169, 224, 224, 3)

In [16]:
x_test.shape

(543, 224, 224, 3)

In [17]:
from keras.applications.vgg16 import VGG16

In [18]:
vgg=VGG16()

In [19]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [20]:
from keras import Sequential

In [21]:
model=Sequential()

In [22]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [24]:
for layer in model.layers:
    layer.trainalbe=False

In [25]:
for layer in model.layers:
  layer.trainable=False

In [26]:
from keras.layers import Dense

In [27]:
model.add(Dense(1,activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [29]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
68/68 [==============================] - 278s 4s/step - loss: 0.5097 - accuracy: 0.7644 - val_loss: 0.3954 - val_accuracy: 0.8379
Epoch 2/5
68/68 [==============================] - 254s 4s/step - loss: 0.3423 - accuracy: 0.8778 - val_loss: 0.3139 - val_accuracy: 0.8729
Epoch 3/5
68/68 [==============================] - 224s 3s/step - loss: 0.2785 - accuracy: 0.8990 - val_loss: 0.2741 - val_accuracy: 0.8932
Epoch 4/5
68/68 [==============================] - 247s 4s/step - loss: 0.2437 - accuracy: 0.9156 - val_loss: 0.2329 - val_accuracy: 0.9171
Epoch 5/5
68/68 [==============================] - 255s 4s/step - loss: 0.2235 - accuracy: 0.9262 - val_loss: 0.2143 - val_accuracy: 0.9245


In [63]:
model_json=model.to_json()
with open("model.json",'w') as file:
    file.write(model_json)

In [64]:
model.save_weights("weight.h5")

In [68]:
from tensorflow.keras.models import model_from_json

with open("model.json","r") as file:
    model_read=file.read()
    
loaded_model=model_from_json(model_read)    

In [69]:
loaded_model.load_weights("weight.h5")

In [75]:
cap=cv2.VideoCapture(0)

In [79]:
while True:
    ret,frame=cap.read()
    

    frame=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(frame)
    
    
    coods=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    if y_pred==0:
        draw_label(frame,"MASK",(30,30),(0,255,0))    
    else:
         draw_label(frame,"NO MASK",(30,30),(0,0,255))
        
        
        
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF==ord('x'):
        break
        
cv2.destroyAllWindows()       

1/1 [==============================] - 0s 130ms/step


In [70]:
def detect_face_mask(img):
    #y_pred=model.predict_classes(img.reshape(1,224,224,3))
    y_pred = (loaded_model.predict(img.reshape(1,224,224,3)) > 0.5)*1
    
    return y_pred[0][0]

In [71]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[0]+text_size[0][1]-2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)
    
    
    

In [77]:
haar=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [74]:
def detect_face(img):
    coods=haar.detectMultiScale(img)
    return coods